Import library

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Read data from kaggle


In [ ]:
data = pd.read_csv("../input/heart-attack-analysis-prediction-dataset/heart.csv")

In [ ]:
data.head()

In [ ]:
data.describe()

Checking duplicate data

In [ ]:
duplicate_datarow = data[data.duplicated()]
print("Duplicate rows: ", duplicate_datarow.shape)

In [ ]:
data = data.drop_duplicates()

In [ ]:
duplicate_datarow = data[data.duplicated()]
print("Duplicate rows: ", duplicate_datarow.shape)

In [ ]:
data.info()

In [ ]:
data.isnull().any()

In [ ]:
data.isnull().sum()

**Basic descriptive analysis**

In [ ]:
data.describe().transpose()

**Exploratory data analysis**

In [ ]:
plt.figure(figsize=(12,7))
sns.countplot(x=data["output"], color="red", alpha=0.4)

In [ ]:
print(data['age'].mean())
print(data['age'].min())
print(data['age'].max())

In [ ]:
plt.figure(figsize=(12,7))
sns.distplot(x=data["age"], bins=15, color="red")

In [ ]:
sns.jointplot(x="age", y="chol", data=data, hue="output")

****Removing out layer****

In [ ]:
data = data[data["chol"] < 380]

In [ ]:
data["chol"].max()

In [ ]:
gender_gb = data.groupby("output")["sex"]
gender_gb.value_counts()

In [ ]:
data = data[data["trtbps"] < 180]

In [ ]:
data["trtbps"].max()

In [ ]:
data = data[data["thalachh"] > 100]

In [ ]:
data["thalachh"].min()

In [ ]:
ekg_normal = len(data[data["restecg"] == 0])/len(data["restecg"])*100
ekg_normal

In [ ]:
ekg1 = len(data[data["restecg"] == 1])/len(data["restecg"])*100
ekg2 = len(data[data["restecg"] == 2])/len(data["restecg"])*100
ekg_normal = len(data[data["restecg"] == 0])/len(data["restecg"])*100

print("Abnormal EKG: {:.2f}".format(ekg1))
print("\n")
print("Hypertrophy by Estes: {:.2f}".format(ekg2))
print("\n")
print("Normal EKG: {:.2f}".format(ekg_normal))

In [ ]:
plt.figure(figsize=(18,7))
sns.heatmap(data.corr(method="pearson"), cmap="PuRd", annot=True, lw=0.1)

In [ ]:
data["caa"] = data["caa"].replace([2,3,4],1)
data["caa"].value_counts()

In [ ]:
data["oldpeak"].min()

In [ ]:
data["oldpeak"] = data["oldpeak"].replace(0.0, 0.01)
data["oldpeak"].value_counts()

In [ ]:
data["log_oldpeak"] = np.log10(data["oldpeak"])

In [ ]:
data = data.drop("oldpeak", axis=1)

In [ ]:
data.corr()["output"]

In [ ]:
data = data.drop(["trtbps", "chol", "restecg", "fbs", "age", "sex"], axis=1)

In [ ]:
data.corr()["output"]

In [ ]:
data.head()

Creating X,Y variable for training

In [ ]:
X = data.drop("output", axis=1)
y = data["output"]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Decision Tree Prediction

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
tree_model = DecisionTreeClassifier(max_depth=15, random_state=42)

In [ ]:
tree_model.fit(X_train, y_train)

In [ ]:
tree_predict = tree_model.predict(X_test)

In [ ]:
tree_predict

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(classification_report(y_test, tree_predict))
print("Confusion Report:")
print(confusion_matrix(y_test, tree_predict))

Random forest classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
random_model = RandomForestClassifier(n_estimators=80, bootstrap=True, random_state=42, criterion="entropy")

In [ ]:
random_model.fit(X_train, y_train)

In [ ]:
random_predict = random_model.predict(X_test)

Predictive mode: Linear model

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
log_model = LogisticRegression(solver="liblinear", random_state=42)

In [ ]:
log_model.fit(X_train, y_train)

In [ ]:
log_predict = log_model.predict(X_test)

In [ ]:
print(classification_report(y_test, log_predict))
print("Confusion Report:")
print(confusion_matrix(y_test, log_predict))

Predictive model Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
gaussian_model = GaussianNB()

In [ ]:
gaussian_model.fit(X_train, y_train)

In [ ]:
gaussian_predict = gaussian_model.predict(X_test)

In [ ]:
print(classification_report(y_test, gaussian_predict))
print("Confusion Report:")
print(confusion_matrix(y_test, gaussian_predict))

All algorithm report

In [ ]:
report = [["GaussianNB", 0.84, 0.84, 0.84, 0.84], ["Random Forest", 0.82, 0.83, 0.82, 0.82], 
          ["DecisionTreeClassifier", 0.81, 0.81, 0.81, 0.81],
          ["LogisticRegression", 0.88, 0.87, 0.86, 0.86]]
overall_result = pd.DataFrame(report, columns=["Model", "Accuracy Score", "Precision", "Recall", "F1-score"])
overall_result.sort_values("Accuracy Score", ascending=False)